In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import os



In [ ]:
def cgr(seq, order, k):
    """
    Creates a two-dimensional numerical representations of the genomic
    sequences using the Chaos Game Representation (CGR)
    :param seq: The String genomic sequence
    :param order: Default order="ACGT"
    :param k: Represents the frequencies of all possible sub-words of length 7
    constructed over the alphabet set {A, C, G, T}.
    :return: 2D plot of CGR
    """
    ln = len(seq)
    pw = 2**k
    out = [[0 for i in range(pw)] for j in range(pw)]
    x = 2**(k-1)
    y = 2**(k-1)

    for i in range(0,ln):
        x=x//2
        y=y//2
        if(seq[i] == order[2] or seq[i] == order[3]):
            x = x + (2**(k-1))
        if(seq[i] == order[0] or seq[i] == order[3]):
            y = y + (2**(k-1))
        if(i>=k-1):
            out[y][x] = out[y][x]+1
    return out

In [ ]:
def create_cgr_sequences(path):
    """
    Reads in every fasta file from directory path paramater. Cleans string then passes it to helper cgr function.
    :param path: The path to read individual fasta files from.
    :return: A list of cgr plots representing each fasta file from original directory
    """
    proper_letters = ["A", "C", "G", "T"]
    os.chdir(path)
    cgr_list_sequences = []
    for file in os.listdir():
        if file.endswith(".fasta"):
            file_path = f"{path}/{file}"
            fasta = SeqIO.read(file_path, "fasta")  # Exception handling occurs in SeqIO.read method
            fasta.upper()

            # Ensure only the letters, A, C, G, T are in sequences
            string_seq = str(fasta.seq)
            for letter in string_seq:
                if letter not in proper_letters:
                    string_seq = string_seq.replace(letter, '')

            # Write this altered sequence back into a SeqRecord object then into separate files
            cgr_seq = cgr(string_seq, "ACGT", 7)

            cgr_list_sequences.append(cgr_seq)
    return cgr_list_sequences


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier


# Update paths to  include Training and Testing Dataset directories
# cwd = os.getcwd()
cwd = "/content/drive/MyDrive/Colab Notebooks/Asn2DataSet/"
training_data_path = os.path.join(cwd, "TrainingDataset")
testing_input = os.path.join(cwd, "TestingDataset")



# Create paths to variant inside TrainingDataset
alpha_input = os.path.join(training_data_path, "Alpha")
beta_input = os.path.join(training_data_path, "Beta")
delta_input = os.path.join(training_data_path, "Delta")
gamma_input = os.path.join(training_data_path, "Gamma")

In [ ]:
# Declare target variable to predict
target_variable = []

# Run each Fasta file through function to clean string then create cgr plot
# Concatenate the 2d plot into a 1d vector and append the variant to target variable list
alpha_sequences = create_cgr_sequences(alpha_input)
for i in range(len(alpha_sequences)):
    alpha_sequences[i] = np.array(alpha_sequences[i]).flatten()
    target_variable.append("Alpha")

beta_sequences = create_cgr_sequences(beta_input)
for i in range(len(beta_sequences)):
    beta_sequences[i] = np.array(beta_sequences[i]).flatten()
    target_variable.append("Beta")

delta_sequences = create_cgr_sequences(delta_input)
for i in range(len(delta_sequences)):
    delta_sequences[i] = np.array(delta_sequences[i]).flatten()
    target_variable.append("Delta")

gamma_sequences = create_cgr_sequences(gamma_input)
for i in range(len(gamma_sequences)):
    gamma_sequences[i] = np.array(gamma_sequences[i]).flatten()
    target_variable.append("Gamma")


# Do the same for testing input without labelling
testing_sequences = create_cgr_sequences(testing_input)
for i in range(len(testing_sequences)):
    testing_sequences[i] = np.array(testing_sequences[i]).flatten()

In [ ]:
# Combine all sequences into one dataframe
all_sequences = alpha_sequences + beta_sequences + delta_sequences + gamma_sequences
X = pd.DataFrame(all_sequences)
X_test = pd.DataFrame(testing_sequences)
# y = pd.DataFrame(target_variable)
y = np.ravel(pd.DataFrame(target_variable)) 


# Normalize the features to by dividing each value by the range ensuring the values fall between 0 and 1.
scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# Ensure length of dataframes match number of input fasta files
print(len(X_scaled)) 
print(len(y))

2000
2000


In [ ]:
print(X_scaled[0])

[0.         0.         0.         ... 0.5        0.         0.28571429]


In [ ]:

# Apply KNN classifier using 10-fold cross validation
knn = KNeighborsClassifier(n_neighbors=3)
scores = cross_val_score(knn, X_scaled, y, cv=10, scoring='accuracy')
mean_accuracy = np.mean(scores)
print("Scores: ")
print(scores)
print("Mean accuracy:", mean_accuracy)


# Predict and predict the labels for the testing set
knn.fit(X_scaled, y)
y_pred = knn.predict(X_test_scaled)
print("Predicted labels:", y_pred)


Scores: 
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Mean accuracy: 1.0
Predicted labels: ['Gamma' 'Gamma' 'Delta' 'Gamma' 'Gamma' 'Delta' 'Beta' 'Beta' 'Delta'
 'Delta']


In [ ]:
from sklearn.linear_model import LogisticRegression


# 10-fold cross-validation with logistic regression
logreg = LogisticRegression(solver='liblinear', multi_class='ovr')
score = cross_val_score(logreg, X_scaled, y, cv=10, scoring='accuracy')
mean_accuracy = np.mean(scores)
print("Scores: ")
print(scores)
print("Mean accuracy:", mean_accuracy)


# Predict the labels for the testing set
logreg.fit(X_scaled, y)
y_pred = logreg.predict(X_test_scaled)
print("Predicted labels:", y_pred)






Scores: 
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Mean accuracy: 1.0
Predicted labels: ['Gamma' 'Gamma' 'Delta' 'Gamma' 'Gamma' 'Delta' 'Beta' 'Beta' 'Delta'
 'Delta']


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=5)
scores = cross_val_score(dtc, X_scaled, y, cv=10, scoring='accuracy')
mean_accuracy = np.mean(scores)

print("Scores: ")
print(scores)
print("Mean accuracy:", mean_accuracy)

# Train the model on the full dataset
dtc.fit(X_scaled, y)
y_pred = dtc.predict(X_test_scaled)
print("Predicted labels:", y_pred)

Scores: 
[1.    1.    0.995 1.    1.    1.    1.    1.    0.995 0.995]
Mean accuracy: 0.9984999999999999
Predicted labels: ['Gamma' 'Gamma' 'Delta' 'Gamma' 'Gamma' 'Delta' 'Beta' 'Beta' 'Delta'
 'Delta']
